In [1]:
import pandas as pd
# from utils import select_min_pay, read_input_file, export_file
import glob
import numpy as np
import pandas as pd
import glob
import logging
import win32com.client as win32
import os
import jaydebeapi
from const import QUERY, CONNECT

In [2]:
Acc = glob.glob("./input/Acc_*.csv")
Acc_date = Acc[0].split('_')[1]

In [3]:
def select_min_pay():
    try:
        # Establish connection to the database
        conn = jaydebeapi.connect(
            CONNECT.DB,
            CONNECT.PORT,
            CONNECT.USER,
            CONNECT.CNN,
        )
        print("Connection established successfully!")

        # Create a cursor to interact with the database
        # cur = conn.cursor()

        # Execute the query and retrieve the result
        print("startquery")
        sql = f'''
    WITH lastdate AS (
SELECT LOAN_NO, MAX(SETL_DT) AS CURRENT_DUE FROM INSTC.CFNC_INDM_L GROUP BY LOAN_NO
)
SELECT 
        a.LOAN_NO, 
        b.CURRENT_DUE,
        NVL(a.INS_THMM_DMND_PCPL_AMT,0) 
        + NVL(a.INS_THMM_DMND_INT_AMT,0) 
        + NVL(a.INS_THMM_DMND_CRED_USAGE_AMT,0)
        + NVL(a.INS_THMM_CLCF_AMT,0)
        + NVL(a.INS_THMM_VAT_AMT,0)  AS LAST_DUE_AMT 
    FROM INSTC.CFNC_INDM_L  a
    INNER JOIN lastdate b 
        ON a.LOAN_NO = b.LOAN_NO 
        AND a.SETL_DT = b.CURRENT_DUE 
    '''
        cur = conn.cursor()
        cur.execute(sql)
        print("Execute")

        # Fetch and convert results to DataFrame
        rows = cur.fetchall()
        col_names = [desc[0] for desc in cur.description]
        cur.close()
        print("close")

        min_pay = pd.DataFrame(rows, columns=col_names)
        print("finishqueryquery")

        return min_pay

    except Exception as e:
        print(f"Error occurred: {e}")
        return None

    # finally:
    #     # Ensure the connection is closed even if an error occurs
    #     if conn:
    #         conn.close()
    #         print("Connection closed.")

In [9]:
MIN_PAY = select_min_pay()

Connection established successfully!
startquery
Execute
close
finishqueryquery


In [10]:
MIN_PAY

,LOAN_NO,CURRENT_DUE,LAST_DUE_AMT
0,9000220022000043,20210917,1641.00
1,9000220022000051,20201117,0.00
2,9000220022000078,20210317,0.00
3,9000220022000086,20210417,0.00
4,9000220022000094,20210317,0.00
...,...,...,...
1879399,7231270101836253,20250301,360.28
1879400,7231270101836585,20250301,1077.94
1879401,7231270101846264,20250301,756.06
1879402,7231270101830597,20250301,1196.33


In [11]:
logging.basicConfig(
    level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s"
)
# Create a logger
logger = logging.getLogger(__name__)


In [12]:
files = glob.glob("./Input/*.*")
files

['./Input\\Acc_20250303_0830.csv',
 './Input\\Code min pay.txt',
 './Input\\GPD SPD.xlsx',
 './Input\\Input min pay.xlsx',
 "./Input\\Map Owner Aug'24 (26082024).xlsx",
 './Input\\New Text Document.txt']

In [13]:
def read_input_file():
    files = glob.glob("./Input/*.*")
    logger.info('Start Processing...')
    logger.info('Reading input files...')
    for file in files:
        if 'Acc' in file:
            logger.info('Reading AR file...')
            AR = pd.read_csv(file, encoding='cp874', low_memory=False)
            AR = AR.rename(columns={'Loan No': 'LOAN_NO'})
            AR['BaseDate'] = AR['BaseDate'].astype(str)
            file_ar = file.replace('.', '_')
            file_lst = file_ar.split('_')
            ar_time = file_lst[3]
        elif 'GPD' in file:
            logger.info('Reading GPD SPD file...')
            GPD_SPD = pd.read_excel(file, engine='openpyxl', dtype={'LOAN_NO': str})
            GPD_SPD = GPD_SPD.rename(columns={'STATUS': 'GPD/SPD'})
        elif 'Map' in file:
            logger.info('Reading Map Owner file...')
            Owner = pd.read_excel(file, engine='openpyxl', dtype={'Loan No.': str})
            Owner = Owner.rename(columns={'Loan No.': 'LOAN_NO', 'Owner': 'OA'})
            Owner = Owner[['LOAN_NO', 'OA', 'Due']]
        else:
            pass
    return AR, ar_time, GPD_SPD, Owner

In [14]:
AR, AR_TIME, GPD_SPD, OWNER = read_input_file()

2025-03-05 10:10:04,998 - INFO - Start Processing...
2025-03-05 10:10:05,000 - INFO - Reading input files...
2025-03-05 10:10:05,000 - INFO - Reading AR file...
2025-03-05 10:10:07,985 - INFO - Reading GPD SPD file...
2025-03-05 10:10:08,469 - INFO - Reading Map Owner file...


In [25]:
AR['LOAN_NO']  = AR['LOAN_NO'].astype('O')

In [29]:
AR = AR.merge(OWNER, on='LOAN_NO', how='left')
AR = AR.merge(GPD_SPD, on='LOAN_NO', how='left')
AR = AR.merge(MIN_PAY, on='LOAN_NO', how='left')


In [30]:
AR

,BaseDate,LOAN_NO,Product Code,Overdue Start Date,OverdueDays_Morning,OverdueCnt_Morning,OverdueDays,OverdueCnt,Loan Amt,Loan Balance,...,OA_x,Due_x,OA_y,Due_y,GPD/SPD_x,OA,Due,GPD/SPD_y,CURRENT_DUE,LAST_DUE_AMT
0,20250303,9010221122000949,A501:TL,2024-10-01,153,6:OD5,153,6:OD5,74164.44,35872.73,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,20250303,9010221122000931,A501:TL,,0,99:NORMAL,0,99:NORMAL,41659.76,12544.04,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,20250303,9010221122000922,A501:TL,2023-08-31,0,98:Write-Off,0,98:Write-Off,34238.99,23851.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,20250303,9010221122000914,A501:TL,2024-02-29,0,98:Write-Off,0,98:Write-Off,16118.33,13559.55,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,20250303,9010221122000906,A501:TL,2022-11-30,0,98:Write-Off,0,98:Write-Off,35235.40,31005.17,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1379249,20250303,1000158090014831,A401:ML,2017-02-01,0,99:NORMAL,0,99:NORMAL,25500.00,100.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1379250,20250303,1000158090014814,A401:ML,2017-08-01,0,99:NORMAL,0,99:NORMAL,24600.00,30.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1379251,20250303,1000158090008580,A401:ML,2017-08-01,0,99:NORMAL,0,99:NORMAL,25900.00,100.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1379252,20250303,1000158090005394,A401:ML,2017-08-01,0,99:NORMAL,0,99:NORMAL,25900.00,100.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [31]:
AR.loc[AR['Product Code'].str.contains('SF', na=False)]

,BaseDate,LOAN_NO,Product Code,Overdue Start Date,OverdueDays_Morning,OverdueCnt_Morning,OverdueDays,OverdueCnt,Loan Amt,Loan Balance,...,OA_x,Due_x,OA_y,Due_y,GPD/SPD_x,OA,Due,GPD/SPD_y,CURRENT_DUE,LAST_DUE_AMT
9956,20250303,7250370104881977,A701:SF,,0,99:NORMAL,0,99:NORMAL,10614.57,10614.57,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9957,20250303,7250370104881971,A701:SF,,0,99:NORMAL,0,99:NORMAL,6623.77,6623.77,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9958,20250303,7250370104881969,A701:SF,,0,99:NORMAL,0,99:NORMAL,3642.96,3642.96,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9959,20250303,7250370104881965,A701:SF,,0,99:NORMAL,0,99:NORMAL,3973.98,3973.98,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9960,20250303,7250370104881959,A701:SF,,0,99:NORMAL,0,99:NORMAL,3002.38,3002.38,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1269761,20250303,7221070100000700,A701:SF,2023-10-31,0,98:Write-Off,0,98:Write-Off,7313.66,5362.75,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1269762,20250303,7221070100000650,A701:SF,2023-10-31,0,98:Write-Off,0,98:Write-Off,10935.41,8018.43,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1269763,20250303,7221070100000641,A701:SF,2023-07-31,0,98:Write-Off,0,98:Write-Off,7064.79,5981.57,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1269764,20250303,7221070100000545,A701:SF,2024-04-30,0,98:Write-Off,0,98:Write-Off,6521.17,519.42,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [35]:
not_map = ['NORMAL', 'Normal', 'normal', 'XD', 'Write-Off']
pattern = '|'.join(not_map)
AR['LAST_DUE_AMT'] = np.where(AR['Product Code'].str.contains('SF', na=False), None, AR['LAST_DUE_AMT'])


In [36]:
AR['LAST_DUE_AMT'] = np.where(AR['OverdueCnt'].str.contains(pattern, na=False), None, AR['LAST_DUE_AMT'])

In [37]:
def set_excel_file_password(excel_file_path, password):
    # Create an Excel application object
    excel_app = win32.Dispatch("Excel.Application")

    # Convert to absolute path
    excel_file_path = os.path.abspath(excel_file_path)

    # Open the Excel file
    wb = excel_app.Workbooks.Open(excel_file_path)

    # Set a password for the entire workbook
    wb.Password = password

    # Save and close the workbook
    wb.Save()
    wb.Close()

In [38]:
AR_SF = AR[AR['Product Code'].str.contains('SF', na=False)].reset_index(drop=True)
AR_ALL = AR[~AR['Product Code'].str.contains('SF', na=False)]
AR_SF1 = AR_SF[:1000001]
AR_SF2 = AR_SF[1000001:]
print('========finishsprit========')
AR_SF_ASSIGN = AR_SF[~AR_SF['OA'].isna()]
logger.info('Exportting AR and AR_SF..')

with pd.ExcelWriter(f'./Output/AR_TOTAL/AR {Acc_date} Time {AR_TIME}.xlsx', engine='xlsxwriter') as writer:
    AR_ALL.to_excel(writer, index=False, sheet_name='AR_ALL')
    print('========finishALL========')
    AR_SF1.to_excel(writer, index=False, sheet_name='AR_SF1')
    print('========finishSF1========') 
    AR_SF2.to_excel(writer, index=False, sheet_name='AR_SF1')
    print('========finishSF2========') 
    AR_SF_ASSIGN.to_excel(writer, index=False, sheet_name='AR_SF_ASSIGN')

2025-03-05 08:52:31,159 - INFO - Exportting AR and AR_SF..


========finishsprit========
========finishALL========
========finishSF1========
========finishSF2========


In [ ]:
AR_SF = AR[AR['Product Code'].str.contains('SF', na=False)].reset_index(drop=True)
AR_ALL = AR[~AR['Product Code'].str.contains('SF', na=False)]
split_point = len(AR_SF) //2  # Splitting roughly in half
AR_SF1 = AR_SF.iloc[:split_point]
AR_SF2 = AR_SF.iloc[split_point:]

In [57]:
max_rows = 500000

# Calculate the number of splits needed
num_splits = (len(AR_SF) // max_rows) + (1 if len(AR_SF) % max_rows > 0 else 0)

# Create a list to store the split DataFrames
split_dataframes = []

# Split the DataFrame into chunks
for i in range(num_splits):
    start_idx = i * max_rows
    end_idx = min((i + 1) * max_rows, len(AR_SF))  # Ensure the last part handles remaining rows
    split_dataframes.append(AR_SF.iloc[start_idx:end_idx])


In [60]:
for idx, df in enumerate(split_dataframes, start=1):
    print(f"DataFrame SF{idx}:")
    print(df)
    print("\n" + "-"*50 + "\n")

DataFrame SF1:
        BaseDate           LOAN_NO Product Code Overdue Start Date  \
0       20240826  7240870103631165      A701:SF                      
1       20240826  7240870103631152      A701:SF                      
2       20240826  7240870103631151      A701:SF                      
3       20240826  7240870103631150      A701:SF                      
4       20240826  7240870103631130      A701:SF                      
...          ...               ...          ...                ...   
499995  20240826  7240370102291249      A701:SF                      
499996  20240826  7240370102291247      A701:SF                      
499997  20240826  7240370102291245      A701:SF                      
499998  20240826  7240370102291242      A701:SF                      
499999  20240826  7240370102291238      A701:SF                      

        OverdueDays_Morning OverdueCnt_Morning  OverdueDays OverdueCnt  \
0                         0          99:NORMAL            0  99:NORMAL

In [ ]:
len(AR_ALL)

In [ ]:
import time
AR_SF = AR[AR['Product Code'].str.contains('SF', na=False)].reset_index(drop=True)
AR_ALL = AR[~AR['Product Code'].str.contains('SF', na=False)]
split_point = len(AR_SF)//2   # Splitting roughly in half
AR_SF1 = AR_SF.iloc[:split_point]
AR_SF2 = AR_SF.iloc[split_point:]

print('========finishsprit========')
AR_SF_ASSIGN = AR_SF[~AR_SF['OA'].isna()]
logger.info('Exportting AR and AR_SF..')
start_time = time.time()
print('========startsave========')

with pd.ExcelWriter(f'./Output/AR_TOTAL/AR {Acc_date} Time {AR_TIME}.xlsx', engine='xlsxwriter') as writer:
    step_start = time.time()
    AR_ALL.to_excel(writer, index=False, sheet_name='AR_ALL')
    print(f'Finished AR_ALL in {time.time() - step_start:.2f} seconds')
    
    step_start = time.time()
    AR_SF1.to_excel(writer, index=False, sheet_name='AR_SF1')
    print(f'Finished AR_SF1 in {time.time() - step_start:.2f} seconds')

    step_start = time.time()
    AR_SF2.to_excel(writer, index=False, sheet_name='AR_SF2')  # Fixed duplicate sheet name
    print(f'Finished AR_SF2 in {time.time() - step_start:.2f} seconds')

    step_start = time.time()
    AR_SF_ASSIGN.to_excel(writer, index=False, sheet_name='AR_SF_ASSIGN')
    print(f'Finished AR_SF_ASSIGN in {time.time() - step_start:.2f} seconds')

    end_time = time.time()
print(f'Total execution time: {end_time - start_time:.2f} seconds')


2025-03-05 09:12:47,428 - INFO - Exportting AR and AR_SF..


========finishsprit========
========startsave========
Finished AR_ALL in 28.29 seconds
Finished AR_SF1 in 73.85 seconds
Finished AR_SF2 in 67.37 seconds
Finished AR_SF_ASSIGN in 11.48 seconds
Total execution time: 181.01 seconds


In [54]:
AR_OA = AR[~AR['OA'].isna()]
OA = AR_OA['OA'].unique()
password = input('กรุณาระบุ password: ')

for owner in OA:
        
    print('========owner========',owner)
    OA_data = AR_OA[AR_OA['OA'] == owner]
    OA_SF = OA_data[OA_data['Product Code'].str.contains('SF', na=False)]
    OA_data = OA_data[~OA_data['Product Code'].str.contains('SF', na=False)]

    
    if OA_SF.shape[0] == 0:
        with pd.ExcelWriter(f'./Output/AR_OA/AR {Acc_date} Time {AR_TIME} {owner}.xlsx', engine='xlsxwriter') as writer:
            OA_data.to_excel(writer, index=False, sheet_name='AR_ALL', engine='xlsxwriter')
    else:
        print(f'OA_data: {len(OA_data)} OA_SF: {len(OA_SF)} ')
        
        with pd.ExcelWriter(f'./Output/AR_OA/AR {Acc_date} Time {AR_TIME} {owner}.xlsx', engine='xlsxwriter') as writer:
            print("startsaveOA_data")
            OA_data.to_excel(writer, index=False, sheet_name='AR_ALL', engine='xlsxwriter')
            print("startsaveOA_SF")
            OA_SF.to_excel(writer, index=False, sheet_name='AR_SF', engine='xlsxwriter') 
    path = f'./Output/AR_OA/AR {Acc_date} Time {AR_TIME} {owner}.xlsx'
    set_excel_file_password(path, password)
logger.info('Exported AR and AR_SF done!!')

========owner======== GMV
========owner======== ACT
========owner======== KOR
OA_data: 3657 OA_SF: 3169 
startsaveOA_data
startsaveOA_SF
========owner======== KWC
========owner======== MGC
OA_data: 4940 OA_SF: 3191 


PermissionError: [Errno 13] Permission denied: './Output/AR_OA/AR 20240826 Time 0830 MGC.xlsx'

In [48]:
AR_OA = AR[~AR['OA'].isna()]
OA = AR_OA['OA'].unique()
password = input('กรุณาระบุ password: ')

for owner in OA:
        
    print('========owner========')
    OA_data = AR_OA[AR_OA['OA'] == owner]
    print('========finishOA_datafitering========')
    OA_SF = OA_data[OA_data['Product Code'].str.contains('SF', na=False)]
    print('========finishOA_SF========')
    OA_data = OA_data[~OA_data['Product Code'].str.contains('SF', na=False)]
    print('========finishOA_data========')
    
    
    if OA_SF.shape[0] == 0:
        with pd.ExcelWriter(f'./Output/AR_OA/AR {Acc_date} Time {AR_TIME} {owner}.xlsx', engine='xlsxwriter') as writer:
            OA_data.to_excel(writer, index=False, sheet_name='AR_ALL', engine='xlsxwriter')
    else:
        with pd.ExcelWriter(f'./Output/AR_OA/AR {Acc_date} Time {AR_TIME} {owner}.xlsx', engine='xlsxwriter') as writer:
            OA_data.to_excel(writer, index=False, sheet_name='AR_ALL', engine='xlsxwriter')
            OA_SF.to_excel(writer, index=False, sheet_name='AR_SF', engine='xlsxwriter') 
    path = f'./Output/AR_OA/AR {Acc_date} Time {AR_TIME} {owner}.xlsx'
    set_excel_file_password(path, password)
logger.info('Exported AR and AR_SF done!!')

========owner========
========owner========
========owner========
========owner========
========owner========


KeyboardInterrupt: 

In [ ]:
def export_file(AR, AR_TIME):
    AR_SF = AR[AR['Product Code'].str.contains('SF', na=False)].reset_index(drop=True)
    AR_ALL = AR[~AR['Product Code'].str.contains('SF', na=False)]
    AR_SF1 = AR_SF[:1000001]
    AR_SF2 = AR_SF[1000001:]
    print('========start========')
    AR_SF_ASSIGN = AR_SF[~AR_SF['OA'].isna()]
    logger.info('Exportting AR and AR_SF..')
    
    with pd.ExcelWriter(f'./Output/AR_TOTAL/AR {Acc_date} Time {AR_TIME}.xlsx', engine='xlsxwriter') as writer:
        step_start = time.time()
        AR_ALL.to_excel(writer, index=False, sheet_name='AR_ALL')
        print(f'Finished AR_ALL in {time.time() - step_start:.2f} seconds')
        
        step_start = time.time()
        AR_SF1.to_excel(writer, index=False, sheet_name='AR_SF1')
        print(f'Finished AR_SF1 in {time.time() - step_start:.2f} seconds')

        step_start = time.time()
        AR_SF2.to_excel(writer, index=False, sheet_name='AR_SF1')  # Fixed duplicate sheet name
        print(f'Finished AR_SF2 in {time.time() - step_start:.2f} seconds')

        step_start = time.time()
        AR_SF_ASSIGN.to_excel(writer, index=False, sheet_name='AR_SF_ASSIGN')
        print(f'Finished AR_SF_ASSIGN in {time.time() - step_start:.2f} seconds')

    end_time = time.time()
            
        
    print('========SAVETOTAL========')
    AR_OA = AR[~AR['OA'].isna()]
    OA = AR_OA['OA'].unique()
    password = input('กรุณาระบุ password: ')
    
    
    print('========STARTLOOP========')
    for owner in OA:
        
        print('========owner========')
        OA_data = AR_OA[AR_OA['OA'] == owner]
        OA_SF = OA_data[OA_data['Product Code'].str.contains('SF', na=False)]
        OA_data = OA_data[~OA_data['Product Code'].str.contains('SF', na=False)]
        if OA_SF.shape[0] == 0:
            with pd.ExcelWriter(f'./Output/AR_OA/AR {Acc_date} Time {AR_TIME} {owner}.xlsx', engine='xlsxwriter') as writer:
                OA_data.to_excel(writer, index=False, sheet_name='AR_ALL', engine='xlsxwriter')
        else:
            with pd.ExcelWriter(f'./Output/AR_OA/AR {Acc_date} Time {AR_TIME} {owner}.xlsx', engine='xlsxwriter') as writer:
                OA_data.to_excel(writer, index=False, sheet_name='AR_ALL', engine='xlsxwriter')
                OA_SF.to_excel(writer, index=False, sheet_name='AR_SF', engine='xlsxwriter') 
        path = f'./Output/AR_OA/AR {Acc_date} Time {AR_TIME} {owner}.xlsx'
        set_excel_file_password(path, password)
    logger.info('Exported AR and AR_SF done!!')

In [39]:
AR_SF = AR[AR['Product Code'].str.contains('SF', na=False)].reset_index(drop=True)
AR_ALL = AR[~AR['Product Code'].str.contains('SF', na=False)]
AR_SF1 = AR_SF[:1000001]
AR_SF2 = AR_SF[1000001:]
print('========start========')
AR_SF_ASSIGN = AR_SF[~AR_SF['OA'].isna()]
print('========finishassign========')
logger.info('Exportting AR and AR_SF..')



2025-03-04 17:29:18,278 - INFO - Exportting AR and AR_SF..


========start========
========finishassign========


In [ ]:
with pd.ExcelWriter(f'./Output/AR_TOTAL/AR {Acc_date} Time {AR_TIME}.xlsx', engine='xlsxwriter') as writer:
    print('========startALL========')
    AR_ALL.to_excel(writer, index=False, sheet_name='AR_ALL', engine='xlsxwriter')
    print('========finishALL========')
    
    
    AR_SF1.to_excel(writer, index=False, sheet_name='AR_SF1', engine='xlsxwriter')
    print('========finishSF1========') 
    AR_SF2.to_excel(writer, index=False, sheet_name='AR_SF1', engine='xlsxwriter')
    print('========finishSF2========') 
    AR_SF_ASSIGN.to_excel(writer, index=False, sheet_name='AR_SF_ASSIGN', engine='xlsxwriter')
    print('========AR_SF_ASSIGN========') 

========startALL========


Exception ignored in: <function ZipFile.__del__ at 0x000001F69F6B8EA0>
Traceback (most recent call last):
  File "c:\Users\Sireetron\AppData\Local\Programs\Python\Python313\Lib\zipfile\__init__.py", line 1980, in __del__
  File "c:\Users\Sireetron\AppData\Local\Programs\Python\Python313\Lib\zipfile\__init__.py", line 1997, in close
ValueError: seek of closed file


========finishALL========


In [33]:
export_file(AR, AR_TIME)

2025-03-04 16:36:08,358 - INFO - Exportting AR and AR_SF..


========start========
========SAVETOTAL========
========STARTLOOP========
========owner========
========owner========
========owner========
========owner========
========owner========
========owner========
========owner========
========owner========
========owner========
========owner========
========owner========
========owner========
========owner========
========owner========
========owner========
========owner========


2025-03-04 17:14:02,544 - INFO - Exported AR and AR_SF done!!
